# pretrain LLM

## 评估生成的文本

### 生成文本

In [ ]:
import torch
from previous_ch import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}



In [5]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

In [8]:
import tiktoken
from previous_ch import generate_text_simple

def text_to_token_ids(text, tokenizer):
    # 为了添加batch维度
    encoded = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

In [9]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

In [16]:
encoded_tensor = text_to_token_ids(start_context, tokenizer)
encoded_tensor


tensor([[6109, 3626, 6100,  345]])

In [12]:
def token_ids_to_text(token_ids, tokenizer):
    flat= tokenizer.decode(token_ids.squeeze(0).tolist())
    return flat

In [18]:
token_ids_to_text(text_to_token_ids(start_context, tokenizer), tokenizer)

'Every effort moves you'

In [21]:
token_ids = generate_text_simple(model = model, idx = encoded_tensor, max_new_tokens=10, context_size=GPT_CONFIG_124M["context_length"])
token_ids_to_text(token_ids, tokenizer)

'Every effort moves you rentingetic wasnم refres RexMeCHicular stren'